Ingredient1: Rank Data
Ingredient2: Song Name Translation
Ingredient3: metadata



In [83]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from piu_utilities import *

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [84]:
csvFolder = "Ranking_Data/"
start = "202307"
end = "202403"
#Read Data month by month
def iterate_year_month_simple(start, end):
    """Iterate from start to end, where start and end are in 'YYYYMM' format."""
    ans = []

    start_year, start_month = divmod(int(start), 100)
    end_year, end_month = divmod(int(end), 100)
    
    current_year, current_month = start_year, start_month
    
    while (current_year, current_month) <= (end_year, end_month):
        ans.append(f'{current_year}{current_month:02d}')  # Format as 'YYYYMM'
        # Increment month
        current_month += 1
        # If month goes beyond December, increment year and reset month to January
        if current_month > 12:
            current_month = 1
            current_year += 1
    return ans

months = iterate_year_month_simple(start, end)

filename = [f"{csvFolder}{month} ranking.csv" for month in months]

# Read in the data

newest_df = pd.read_csv(filename[len(filename)-1])


In [85]:
#make initial dataframe from songList

Songlist_file = "Metadata-ComposerSongNames.txt"

with open(Songlist_file, encoding = "utf-8") as f:
    songList = f.readlines()

kr_songList = []
kr_composerList = []
en_songList = []
en_composerList = []
for item in songList:
    item = item.split("//")
    kr_songList.append(item[0])
    kr_composerList.append(item[1])
    en_songList.append(item[2])
    en_composerList.append(item[3][:-1])

    
songList_df = pd.DataFrame({'song':kr_songList, 'composer':kr_composerList})
                           # , 'en_song':en_songList, 'en_composer':en_composerList})

songList_df


,song,composer
0,스테이저,Ras
1,리틀 먼치킨,Castellia
2,서든 어페어런스 이미지,Blacklolita
3,톰보이,(여자)아이들
4,"사이먼 세이즈, 유로댄스!! (feat. Sara☆M)",Jehezukiel☆KAZEMURA☆Hideky
...,...,...
539,눈을 감아,반야
540,미운 오리 토카타,반야 프로덕션
541,셋업 미투 믹스,반야 프로덕션
542,유니크 - SHORT CUT -,SHK


In [86]:
#결측값 보정
newest_df['song'] = newest_df['song'].apply(correct_anomalies_custom)

hello()

'hello'

In [87]:
#if there are elements of the songList that are not in the newest_df, print them
def check_differences(df1, df2, column1, column2):
    print(f"Elements in {column1} in df1 that are not in df2")
    for item in df1[column1]:
        if item not in df2[column2].values:
            print(item)
    print(" ")
    print(f"Elements in {column2} in df2 that are not in df1")
    for item in df2[column2]:
        if item not in df1[column1].values:
            print(item)

check_differences(songList_df, newest_df, 'kr_song', 'song')


Elements in kr_song in df1 that are not in df2


KeyError: 'kr_song'

In [88]:
import copy
newest_df_copy = copy.deepcopy(newest_df)

newest_df_copy = newest_df_copy.merge(songList_df, left_on = 'song', right_on = 'kr_song', how = 'left')

KeyError: 'kr_song'

In [89]:
newest_df_copy

,rank,song,pattern
0,1,애프터 라이크,E01
1,2,애프터 라이크,N04
2,3,톰보이,N03
3,4,애프터 라이크,H06
4,5,톰보이,H05
...,...,...,...
3833,3834,미션 파서블,S05
3834,3835,마이 드림즈,S04
3835,3836,알레그로 피유 모쏘,S04
3836,3837,아크 오브 다크니스,S02


In [90]:
# Step 1: Add a new column 'pattern_list' to songList_df with empty lists

songList_df['pattern_list'] = [[] for _ in range(len(songList_df))]
songList_df['rank_list'] = [[] for _ in range(len(songList_df))]
# Step 2: Iterate through newest_df_copy and update 'pattern_list' in songList_df
for _, row in newest_df_copy.iterrows():
    # Find the index in songList_df where the song name matches
    index = songList_df[songList_df['song'] == row['song']].index
    
    # Append the pattern to the pattern_list of the matching song
    for i in index:
        songList_df.at[i, 'pattern_list'].append(row['pattern'])
        songList_df.at[i, 'rank_list'].append(row['rank'])

# Verify the results


In [ ]:
#sort pattern_list and rank_list accordingly. Sorting order: E N H V S D C
def sort_pattern_rank_lists(pattern_list, rank_list):

    pattern_list = [x for _, x in sorted(zip(rank_list, pattern_list))]
    rank_list.sort()
    return pattern_list, rank_list



In [95]:
for i in range(len(songList_df)):
    songList_df.at[i, 'pattern_list'], songList_df.at[i, 'rank_list'] = sort_pattern_rank_lists(songList_df.at[i, 'pattern_list'], songList_df.at[i, 'rank_list'])

songList_df

,song,composer,pattern_list,rank_list
0,스테이저,Ras,"[S17, S19, D20, D18, S15, D16, S11, C, D12, S08]","[57, 58, 59, 109, 148, 174, 289, 364, 405, 714]"
1,리틀 먼치킨,Castellia,"[S16, S19, D23, D20, S21, D17, D15, S13, S08, ...","[89, 100, 101, 105, 120, 135, 173, 190, 330, 3..."
2,서든 어페어런스 이미지,Blacklolita,"[S17, D23, S19, S21, D21, D17, S15, S13, S08]","[98, 103, 112, 115, 121, 123, 162, 235, 666]"
3,톰보이,(여자)아이들,"[N03, H05, V08, S08, S17, D22, S19, S05, D18, ...","[3, 5, 15, 62, 87, 108, 132, 143, 145, 158, 18..."
4,"사이먼 세이즈, 유로댄스!! (feat. Sara☆M)",Jehezukiel☆KAZEMURA☆Hideky,"[S18, S20, S16, D22, D19, D13, S11, S07, S04]","[90, 93, 99, 106, 110, 216, 265, 747, 1016]"
...,...,...,...,...
539,눈을 감아,반야,[S06],[3546]
540,미운 오리 토카타,반야 프로덕션,"[D18, S17]","[3595, 3658]"
541,셋업 미투 믹스,반야 프로덕션,[S18],[3492]
542,유니크 - SHORT CUT -,SHK,"[D16, S18]","[3550, 3701]"
